In [121]:
import pandas as pd

In [122]:
bandwidth = pd.read_excel('ToyData.xlsx',sheet_name='Inter-Datacenter Links')
dcd = pd.read_excel('ToyData.xlsx',sheet_name='Data Center Details')
slot = dcd.loc[:,['DC','Num of Slots']].dropna(how='any')
location = dcd.loc[:,['Data Partition','Location']]

In [132]:
RQ = pd.read_excel('ToyData.xlsx',sheet_name='Job List').iloc[:,1:]
RQ.set_index(list(RQ)[0],inplace=True)
pre = RQ[list(RQ)[-1]].dropna()
exe = RQ[list(RQ)[-2]]
RQ = RQ.iloc[:,0:-2]

In [133]:
pre = pre.apply(lambda x: x.replace("{","").replace("}",""))

In [135]:
pre.str.split(", ",expand=True)

,0,1,2,3,4,5,6,7,8,9,10
Unnamed: 1,,,,,,,,,,,
tA1,,None,None,None,None,None,None,None,None,None,None
tB1,"(tB1,tB2)",None,None,None,None,None,None,None,None,None,None
tC1,"(tC1,tC2)","(tC1,tC3)","(tC2,tC3)",None,None,None,None,None,None,None,None
tD1,"(tD1,tD3)","(tD2,tD3)","(tD2,tD4)","(tD3,tD5)",None,None,None,None,None,None,None
tE1,"(tE1,tE2)","(tE1,tE3)","(tE2,tE4)","(tE3,tE5)","(tE3,tE6)","(tE5,tE6)",None,None,None,None,None
tF1,"(tF1,tF2)","(tF1,tF3)","(tF1,tF4)","(tF2,tF5)","(tF3,tF5)","(tF4,tF6)","(tF5,tF7)","(tF5,tF8)","(tF6,tF8)","(tF7,tF9)","(tF8,tF9)"
